In [46]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os

import pandas as pd
import numpy as np
import math

# Inputs
This file generates http://e4e.ucsd.edu/people and http://e4e.ucsd.edu/alumni from https://docs.google.com/spreadsheets/d/1toCpXPNdHbC1qH6K88a6xDdoVQr2X0aHA4tG5DyQuj0/edit?usp=sharing

The Level column should be set to one of the following:

| Value | Level                          |
|-------|--------------------------------|
| 1     | Project Candidate              |
| 2     | Technical Contributor          |
| 3     | Research Contributor           |
| 4     | Project Lead                   |
| 5     | Staff Engineer                 |
| 6     | E4E Director                   |
| 22    | Technical Contributor - Alumni |
| 23    | Research Contributor - Alumni  |
| 24    | Project Lead - Alumni          |
| 25    | Former Staff Engineer          |
| 26    | Former E4E Director            |

This should represent the highest level achieved by the person throughout their time in E4E.  For example, if a project lead decides to step back and let someone else be a project lead while still contributing to the project, for the purposes of this, they will retain the level of project lead.

The Expeditionary column should be set to either True or False, denoting whether that person has been a part of any expeditions.

The Title column should be filled with that person's full name (Title, first, last, suffix).

If an image exists, it should be uploaded to the E4E website, and the link to it placed in the Image section.  Nominally, these images should be 200x200 px.  If possible, name these as follows: `f"{year}_{first_initial}{last_name}_{width}x{height}.{extension}"`

The Description column should be a concise description of that person's project affiliation.  Each project affiliation should be on a separate line, and consist of the project name followed by the timespan of the affiliation.  If the student was only a part of the summer REU program, then the timespan should be denoted as `f"{year} Summer REU"`.  If the student was involved for at least a quarter, then simply put the calendar year.  So if someone was involved during Fall 2021 and Winter 2022, the timespan should be denoted as `2021-2022`.

If known, enter the End Year and Start Year in the respective columns.  These should be the calendar start/end years.

If desired, a link to the person's personal webpage can be added under Link.

# Outputs
The contents of `people.txt` can be pasted directly into http://e4e.ucsd.edu/wp-admin/post.php?post=5303&action=edit

The contents of `alumni.txt` can be pasted directly into http://e4e.ucsd.edu/wp-admin/post.php?post=5354&action=edit


In [47]:
expeditionsSheet_ID = '1toCpXPNdHbC1qH6K88a6xDdoVQr2X0aHA4tG5DyQuj0'
expeditionsDataRange = 'Sheet1'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [48]:
creds = None
expCredsPath = 'expeditions.pkl'
if os.path.exists(expCredsPath):
    with open(expCredsPath, 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open(expCredsPath, 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId = expeditionsSheet_ID,
                           range=expeditionsDataRange).execute()
values = result.get('values', [])

In [49]:
def standardHTML(name, desc, img, expedition, link):
    title = name
    altTitle = name
    imageURL = img
    description = desc
    html = ""
    if link == "":
        link = None
    
    if expedition:
        title += "*"
    html += "<!-- wp:html -->\n"
    html += "<div class=\"personnel\">\n"
    html += '<p><img class="profileimg" title="%s" src="%s" alt="%s"></p>\n' % (altTitle, imageURL, altTitle)
    if link:
        html += '<p class="projecttitle"><a href="%s">%s</a></p>\n' % (link, title)
    else:
        html += '<p class="projecttitle">%s</p>\n' % (title)
    html += '<p class="projectdescription">%s</p>\n' % (description)
    html += '</div>\n'
    html += '<!-- /wp:html -->\n'
    html += '\n'
    return html

def minimalHTML(name, desc, img, expedition, link):
    title = name
    html = ""
    if expedition:
        title += "*"
    
    html += '<li>%s - %s</li>\n' % (title, desc)
    return html

def noOutput(name, desc, img, expedition, link):
    return ""

In [50]:
baseLevels = {
    1: "Project Candidate",
    2: "Technical Contributor",
    3: "Research Contributor",
    4: "Project Lead",
    5: "Staff Engineer",
    6: "E4E Director",
    22: "Technical Contributor - Alumni",
    23: "Research Contributor - Alumni",
    24: "Project Lead - Alumni",
    25: "Former Staff Engineer",
    26: "Former E4E Director"
}

expeditionLevels = {
    3: "Expedition Member",
    4: "Expedition Student Leader",
    5: "Staff Engineer",
    6: "E4E Director",
    23: "Expedition Alumni",
    24: "Expedition Leader Alumni",
    25: "Former Staff Engineer",
    26: "Former E4E Director"
}

baseLevelHeadings = {
    1: "Project Candidates",
    2: "Technical Contributors",
    3: "Research Contributors",
    4: "Project Leads",
    5: "Staff Engineers",
    6: "E4E Directors",
    22: "Technical Contributor - Alumni",
    23: "Research Contributor - Alumni",
    24: "Project Lead - Alumni",
    25: "Former Staff Engineers",
    26: "Former E4E Directors"
}

levelFormatter = {
    1: noOutput,
    2: minimalHTML,
    3: standardHTML,
    4: standardHTML,
    5: standardHTML,
    6: standardHTML,
    22: minimalHTML,
    23: standardHTML,
    24: standardHTML,
    25: standardHTML,
    26: standardHTML
}

sectionOutput = {
    1: ("", ""),
    2: ("<!-- wp:list -->\n<ul>\n", "</ul>\n<!-- /wp:list -->\n"),
    3: ("", ""),
    4: ("", ""),
    5: ("", ""),
    6: ("", ""),
    22: ("<!-- wp:list -->\n<ul>\n", "</ul>\n<!-- /wp:list -->\n"),
    23: ("", ""),
    24: ("", ""),
    25: ("", ""),
    26: ("", "")
}

peoplePageOrder = [
    6, 5, 4, 3, 2
]

alumniPageOrder = [
    26, 25, 24, 23, 22
]

blankProfilePic = "http://e4e.ucsd.edu/wp-content/uploads/blank-profile-drawing.png"

In [51]:
df = pd.DataFrame(values[1:], columns=values[0])

In [52]:
df[["Level", "End year", "Start year"]] = df[["Level", "End year", "Start year"]].apply(pd.to_numeric)
df['Expeditionary'] = df['Expeditionary'].map({"TRUE": True, "FALSE": False})
df['Image'] = df['Image'].replace(['', None], [blankProfilePic, blankProfilePic])
df['End year'].fillna(9999, inplace=True)
df["Link"].fillna(value="", inplace=True)
df["Description"].fillna(value="", inplace=True)

df.sort_values(['Level', 'End year', 'Start year'], inplace=True, ascending=[True, False, False])

In [53]:
html = ''
html += "<!-- wp:paragraph -->\n"
html += "<p>* Denotes people that have participated in one of our <a href=\"http://e4e.ucsd.edu/expeditions\">expeditions/deployments</a>.</p>\n"
html += "<!-- /wp:paragraph -->\n"
html += "\n"
for level in peoplePageOrder:
    people = df[df['Level'] == level]
    if len(people) > 0:
        html += "<!-- wp:heading -->\n"
        html += "<h2>%s</h2>\n" % (baseLevelHeadings[level])
        html += "<!-- /wp:heading -->\n"
        html += "\n"

        html += sectionOutput[level][0]
    
    for personIdx in people.index:
        if people.loc[personIdx, 'Start year'] == 9999:
            print(f'Warning: {people.loc[personIdx, "Title"]} does not have a defined end year!')
        title = people.loc[personIdx, "Title"]
        altTitle = people.loc[personIdx, "Title"]
        imageURL = people.loc[personIdx, "Image"]
        description = people.loc[personIdx, "Description"]
        expedition = people.loc[personIdx, "Expeditionary"]
        link = people.loc[personIdx, "Link"]
        html += levelFormatter[level](title, description, imageURL, expedition, link)
        
    html += sectionOutput[level][1]
with open("people.txt", 'w') as file:
    file.write(html)

In [54]:
html = ''
html += "<!-- wp:paragraph -->\n"
html += "<p>* Denotes people that have participated in one of our <a href=\"http://e4e.ucsd.edu/expeditions\">expeditions/deployments</a>.</p>\n"
html += "<!-- /wp:paragraph -->\n"
html += "\n"
for level in alumniPageOrder:
    people = df[df['Level'] == level]
    if len(people) > 0:
        html += "<!-- wp:heading -->\n"
        html += "<h2>%s</h2>\n" % (baseLevelHeadings[level])
        html += "<!-- /wp:heading -->\n"
        html += "\n"

        html += sectionOutput[level][0]
    
    for personIdx in people.index:
        if people.loc[personIdx, 'End year'] == 9999:
            print(f'Warning: {people.loc[personIdx, "Title"]} does not have a defined end year!')
        title = people.loc[personIdx, "Title"]
        altTitle = people.loc[personIdx, "Title"]
        imageURL = people.loc[personIdx, "Image"]
        description = people.loc[personIdx, "Description"]
        expedition = people.loc[personIdx, "Expeditionary"]
        link = people.loc[personIdx, "Link"]
        html += levelFormatter[level](title, description, imageURL, expedition, link)
        
    html += sectionOutput[level][1]
with open("alumni.txt", 'w') as file:
    file.write(html)